In [ ]:
!pip install pulp
from pulp import *
import pandas as pd
import numpy as np

In [ ]:
amount = 140
surgeries = [x for x in range(0,amount)]
block_numbers = [x for x in range(0,32)]
positions = [x for x in range(0,15)]
blocks = {
            0:{"category":1,"day":1},
            1:{"category":2,"day":1},
            2:{"category":3,"day":1},
            3:{"category":4,"day":1},
            4:{"category":5,"day":1},
            5:{"category":2,"day":1},
            6:{"category":5,"day":1},
            7:{"category":1,"day":2},
            8:{"category":3,"day":2},
            9:{"category":3,"day":2},
            10:{"category":4,"day":2},
            11:{"category":4,"day":2},
            12:{"category":5,"day":2},
            13:{"category":1,"day":3},
            14:{"category":1,"day":3},
            15:{"category":2,"day":3},
            16:{"category":6,"day":3},
            17:{"category":4,"day":3},
            18:{"category":4,"day":3},
            19:{"category":5,"day":3},
            20:{"category":3,"day":3},
            21:{"category":1,"day":4},
            22:{"category":3,"day":4},
            23:{"category":4,"day":4},
            24:{"category":4,"day":4},
            25:{"category":5,"day":4},
            26:{"category":1,"day":5},
            27:{"category":2,"day":5},
            28:{"category":3,"day":5},
            29:{"category":4,"day":5},
            30:{"category":5,"day":5},
            31:{"category":2,"day":5},   
        }
def create_ops(number_of_surgeries):
  op_informations = {
                  1:{"mean":132,"std":76,"ratio":0.18},
                  2:{"mean":99,"std":53,"ratio":0.14},
                  3:{"mean":142,"std":58,"ratio":0.17},
                  4:{"mean":78,"std":52,"ratio":0.28},
                  5:{"mean":72,"std":38,"ratio":0.18},
                  6:{"mean":75,"std":72,"ratio":0.05}
              }
  surgery_blocks_for_surgery = {}
  duration_of_surgery = {}
  last_element = 0
  for op in range(1,7):
    amount_of_category = round(amount*op_informations[op]["ratio"])
    for x in range(last_element,last_element+amount_of_category):
      if last_element>=amount:
        continue
      length_of_Op = np.random.normal(op_informations[op]["mean"],op_informations[op]["std"],amount_of_category)
      duration_of_surgery[last_element] = int(abs(length_of_Op[x-last_element]))
      selectable_blocks = [key for key in blocks.keys() if blocks[key]["category"] == op]
      surgery_blocks_for_surgery[last_element] = selectable_blocks
      last_element +=1
  return duration_of_surgery, surgery_blocks_for_surgery

duration_of_surgeries, block_for_surgery = create_ops(len(surgeries))
duration_of_surgeries

In [ ]:
import random
def create_emergencies(number_of_scenarios):
  emergencies = {}
  blocks_for_emergencies = {}
  for scenario in range(number_of_scenarios):
    emergencies[scenario] = {}
    blocks_for_emergencies[scenario] = {}
    counter=0
    for day in range(1,6):
      number_of_emergencies = random.randint(1, 3)
      for em in range(number_of_emergencies):
        emergencies[scenario][counter] = random.randint(60,180)
        blocks_for_emergencies[scenario][counter] = [_ for _ in blocks if blocks[_]["day"]==day]
        counter +=1
  return emergencies, blocks_for_emergencies

emergencies,blocks_for_emergencies = create_emergencies(2)    


In [ ]:
prob = LpProblem("operationPlan",LpMinimize)

In [ ]:
set_surgery = LpVariable.dicts("y_ibp",[(i,b,p) for i in surgeries for b in block_for_surgery[i] for p in positions],cat="Binary")
preschdeuled_Start = LpVariable.dicts("t_bp",(block_numbers,positions),lowBound=0)
emergencySet = LpVariable.dicts("zS_ebp",[(s,e,b,p) for s in emergencies.keys() for e in emergencies[s] for b in blocks_for_emergencies[s][e] for p in positions],cat="Binary")
surgery_canceled = LpVariable.dicts("xs_ibp", [(s,i,b,p) for s in emergencies.keys() for i in surgeries for b in block_for_surgery[i] for p in positions],cat="Binary")
actual_start =  LpVariable.dicts("t_hat_s_bp",(emergencies.keys(),block_numbers,positions),lowBound=0)
waiting_time = LpVariable.dicts("ws_i",[(s,i) for s in emergencies.keys() for i in surgeries],lowBound=0)
overtime = LpVariable.dicts("os_b",[(s,b) for s in emergencies.keys() for b in blocks.keys()],lowBound=0)
idle_time =  LpVariable.dicts("gs_b",[(s,b) for s in emergencies.keys() for b in blocks.keys()],lowBound=0)

In [ ]:
prob += lpSum([1*set_surgery[(i,b,p)] for i in surgeries for b in block_for_surgery[i] for p in positions]) + lpSum([100*(1-lpSum([set_surgery[(i,b,p)] for b in block_for_surgery[i] for p in positions])) for i in surgeries]) + (1/len(emergencies.keys()) * lpSum([(lpSum([2*overtime[(s,b)]+ 0.5 *idle_time[(s,b)] for b in block_numbers]) + lpSum([1*waiting_time[(s,i)]+(300*lpSum([surgery_canceled[(s,i,b,p)] for b in block_for_surgery[i] for p in positions]))for i in surgeries])) for s in emergencies.keys()]))

In [ ]:

#constraints
M = 9999999
#2
for i in surgeries:
  prob += (lpSum([set_surgery[(i,b,p)]for b in block_for_surgery[i] for p in positions]) <= 1,f"2_{i}")
#3
for b in block_numbers:
  for p in positions:
    prob += (lpSum([set_surgery[(i,b,p)] for i in surgeries if b in block_for_surgery[i]]) <=1,f"3_{b}_{p}")
#4
for s in emergencies.keys():
  for e in emergencies[s]:
    prob += (lpSum([emergencySet[(s,e,b,p)] for b in blocks_for_emergencies[s][e] for p in positions]) == 1,f"4_{s}_{e}")
#5
for b in block_numbers:
  for p in positions:
    for s in emergencies.keys():
      prob +=( actual_start[s][b][p] >= preschdeuled_Start[b][p],f"5_{b}_{p}_{s}")
#6
for b in block_numbers:
  for s in emergencies.keys():
    for p in positions:
      if p!=0:
        prob += (actual_start[s][b][p] >= actual_start[s][b][p-1] + lpSum([duration_of_surgeries[i]*(set_surgery[(i,b,p-1)]-surgery_canceled[(s,i,b,p-1)]) for i in surgeries if b in block_for_surgery[i]]) + lpSum([emergencies[s][e]* emergencySet[(s,e,b,p)] for e in blocks_for_emergencies[s].keys() if b in blocks_for_emergencies[s][e]] ),f"6_{b}_{s}_{p}")
#7
for b in block_numbers:
  for s in emergencies.keys():
    prob += (actual_start[s][b][0] >= lpSum([emergencies[s][e] * emergencySet[(s,e,b,0)] for e in emergencies[s].keys() if b in blocks_for_emergencies[s][e]]),f"7_{b}_{s}")
#8
for i in surgeries:
  for b in block_for_surgery[i]:
    for p in positions:
      for s in emergencies.keys():
        prob += (waiting_time[(s,i)] >= actual_start[s][b][p] - preschdeuled_Start[b][p] - M * (1-(set_surgery[(i,b,p)] - surgery_canceled[(s,i,b,p)])),f"8_{i}_{b}_{p}_{s}")
#9
for b in block_numbers:
  for s in emergencies.keys():
    prob += (idle_time[(s,b)] == lpSum([actual_start[s][b][p] - (actual_start[s][b][p-1] + lpSum([duration_of_surgeries[i]*(set_surgery[(i,b,p-1)]- surgery_canceled[(s,i,b,p-1)]) for i in surgeries if b in block_for_surgery[i]]) + lpSum([ emergencies[s][e] * emergencySet[(s,e,b,p)] for e in emergencies[s] if b in blocks_for_emergencies[s][e]]))for p in range(1,len(positions)) ]),f"9_{b}_{s}")

#10
for b in block_numbers:
  for s in emergencies.keys():
    prob += (0>= (actual_start[s][b][len(positions)-1] + lpSum([duration_of_surgeries[i]*(set_surgery[(i,b,len(positions)-1)]-surgery_canceled[(s,i,b,len(positions)-1)])for i in surgeries if b in block_for_surgery[i]]) + lpSum([emergencies[s][e] * emergencySet[(s,e,b,len(positions)-1)] for e in emergencies[s] if b in  blocks_for_emergencies[s][e] ]))-480,f"10_{b}_{s}")

#11
for i in surgeries:
  for b in block_for_surgery[i]:
    for p in positions:
      prob+= (lpSum([surgery_canceled[(s,i,b,p)] for s in emergencies.keys()]) <= len(emergencies.keys()) * set_surgery[(i,b,p)],f"11_{i}_{b}_{p}")

for b in block_numbers:
  for p in positions:
    if p>0:
      prob += (preschdeuled_Start[b][p] == preschdeuled_Start[b][p-1] + lpSum([duration_of_surgeries[i]*(set_surgery[(i,b,p-1)] - surgery_canceled[(s,i,b,p-1)]) for i in surgeries if b in block_for_surgery[i]]),f"12_own_{b}_{p}")

for b in block_numbers:
    prob+= (preschdeuled_Start[b][0] ==0)


In [ ]:
prob.solve(PULP_CBC_CMD(msg=True, timeLimit=1200))

Export

In [ ]:
print("Status:", LpStatus[prob.status])
count = 0 
names = []
print()
for v in prob.variables():
      if v.varValue != 0 and "y_ibp_(" in v.name:
        names.append(v.name)
        count+=1
        print(count)
        #v.fixValue()
        print(v.name, "=", v.varValue)

names.sort()
print(names)


print("Status:", LpStatus[prob.status])
count = 0 
em = []
print(prob.variables()[1])
for v in prob.variables():
      if v.varValue != 0 and "z" in v.name:
        em.append(v)
        count+=1
        print(count)
        #v.fixValue()
        print(v.name, "=", v.varValue)

In [ ]:
import re
table = pd.DataFrame(np.full((15,32),"-1"))
table
for b in range(0,32):
  for p in range(0,15):
    for i in range(0,140):
      for _ in names:
        if f"y_ibp_({i},_{b},_{p})" == _ :
          for v in prob.variables():
            if f"t_hat_s_bp_0_{b}_{p}" == v.name:
              print(table.loc[p,b])
              if table.loc[p,b] != "-1":
                print("!")
                table.at[p,b] =table.loc[p,b]+ " | "+ str(i) + "  " +str(int(v.varValue))
              else:
                table.at[p,b] = str(i) + "  " +str(int(v.varValue))
              #print(p, "  ",b ,"  ", i)
              #table.at[p,b] = str(i) + "  " +str(int(v.varValue))
    for e in emergencies[0].keys():
      for v in em:
        if f"zS_ebp_(0,_{e},_{b},_{p})" == v.name:
          for v_ in prob.variables():
            if f"t_hat_s_bp_0_{b}_{p}" == v_.name:
              print("table.loc[p,b]")
              if table.loc[p,b] != "-1":
                print("!")
                table.at[p,b] = table.loc[p,b]+"|  z "+str(e) + " ("+ str(int(v_.varValue)) + "  "+ v_.name +") " +str(int(v_.varValue)- emergencies[0][e])
              else:
                table.at[p,b] = "z "+str(e) + "  " +" ("+str(int(v_.varValue))+ "  "+v_.name+") "+str(int(v_.varValue)- emergencies[0][e])

In [ ]:
table

In [ ]:
table.to_csv("ex.csv")

Kennzahlen


In [ ]:
#gesamte zeit
sum(duration_of_surgeries.values()) +(sum(emergencies[0].values()) +sum(emergencies[1].values()))/2 

14612.5

In [ ]:
# notfälle
count =0
duration_added_emergencies = 0
print("Status:", LpStatus[prob.status])
for v in prob.variables():
       if v.varValue !=0 and "zS_ebp" in v.name:
          duration_added_emergencies += emergencies[int(re.findall(r'\d{1,2}',v.name)[0])][int(re.findall(r'\d{1,2}',v.name)[1])]
          print(v.name, "=", v.varValue)
          count += 1
print(count)
print(duration_added_emergencies/2)

In [ ]:
# Cancell
count =0
print("Status:", LpStatus[prob.status])
for v in prob.variables():
       if v.varValue !=0 and "xs_ibp" in v.name:
          print(v.name, "=", v.varValue)
          count+=1
print(count)

In [ ]:
# Operationen
print("Status:", LpStatus[prob.status])
duration_added_surg = 0
count = 0
for v in prob.variables():
       if v.varValue !=0 and "y_ibp" in v.name:
          duration_added_surg += duration_of_surgeries[int(re.findall(r'\d{1,2}',v.name)[0])]
          print(v.name, "=", v.varValue)
          count +=1
print(count)
print(duration_added_surg)

In [ ]:
# Wartezeit
count = 0
print("Status:", LpStatus[prob.status])
for v in prob.variables():
       if v.varValue !=0 and "wi_s" in v.name:
          print(v.name, "=", v.varValue)
          count +=1
print(count)

In [ ]:
duration_added_emergencies/2 + duration_added_surg